### Experiment with multiple variables

In [ ]:
import sys
sys.path.append('../')
from src.sampling import sample_from_distribution
from src import Game,BackwardInductionSolver
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

In [ ]:
player_1 = "China" 
player_2 = "US" 

### TT Data

In [ ]:
player1_tt = {"tt": {"var1": {"mean":-1,
                              "stdev":1
                             },
                     "var2": {"mean":-1,
                              "stdev":1
                             },
                     "var3": {"mean":-1,
                              "stdev":1
                             },
                     "var4": {"mean":-1,
                              "stdev":1
                             },
                     "var5": {"mean":-1,
                              "stdev":1
                             },
                    }
             }

player2_tt = {"tt": {"var1": {"mean":-1,
                              "stdev":1
                             },
                     "var2": {"mean":-1,
                              "stdev":1
                             },
                     "var3": {"mean":-1,
                              "stdev":1
                             },
                     "var4": {"mean":-1,
                              "stdev":1
                             },
                     "var5": {"mean":-1,
                              "stdev":1
                             },
                    }
             }

### TNT Data

In [ ]:
player1_tnt = {"tnt": {"var1": {"mean":1,
                                "stdev":1
                               },
                       "var2": {"mean":-1,
                                "stdev":1
                               },
                       "var3": {"mean":1,
                                "stdev":1
                               },
                       "var4": {"mean":-1,
                                "stdev":1
                               },
                       "var5": {"mean":0,
                              "stdev":1
                             },
                      }
              }

player2_tnt = {"tnt": {"var1": {"mean":-2,
                                "stdev":1
                               },
                       "var2": {"mean":-2,
                                "stdev":1
                               },
                       "var3": {"mean":-2,
                                "stdev":1
                               },
                       "var4": {"mean":-2,
                                "stdev":1
                               },
                       "var5": {"mean":-2,
                              "stdev":1
                             },
                      }
              }

### NTT Data

In [ ]:
player1_ntt = {"ntt": {"var1": {"mean":-2,
                                "stdev":1
                               },
                       "var2": {"mean":-2,
                                "stdev":1
                               },
                       "var3": {"mean":-2,
                                "stdev":1
                               },
                       "var4": {"mean":-2,
                                "stdev":1
                               },
                       "var5": {"mean":-2,
                              "stdev":1
                             },
                      }
              }

player2_ntt = {"ntt": {"var1": {"mean":1,
                                "stdev":1
                               },
                       "var2": {"mean":-1,
                                "stdev":1
                               },
                       "var3": {"mean":1,
                                "stdev":1
                               },
                       "var4": {"mean":-1,
                                "stdev":1
                               },
                       "var5": {"mean":0,
                              "stdev":1
                             },
                      }
              }

### NTNT Data

In [ ]:
player1_ntnt = {"ntnt": {"var1": {"mean":-1,
                                "stdev":1
                               },
                       "var2": {"mean":-1,
                                "stdev":1
                               },
                       "var3": {"mean":-1,
                                "stdev":1
                               },
                       "var4": {"mean":-1,
                                "stdev":1
                               },
                       "var5": {"mean":-1,
                              "stdev":1
                             },
                      }
              }

player2_ntnt = {"ntnt": {"var1": {"mean":-1,
                                "stdev":1
                               },
                       "var2": {"mean":-1,
                                "stdev":1
                               },
                       "var3": {"mean":-1,
                                "stdev":1
                               },
                       "var4": {"mean":-1,
                                "stdev":1
                               },
                       "var5": {"mean":-1,
                              "stdev":1
                             },
                      }
              }

In [ ]:
player1_data = player1_tt | player1_tnt | player1_ntt | player1_ntnt
player2_data = player2_tt | player2_tnt | player2_ntt | player2_ntnt

In [ ]:
player1_data

In [ ]:
class Player(): 
    def __init__(self, data, player_name):
        self.data = data 
        self.name = player_name

    def calculate_payoff(self, scenario, payoff_func):
        # To Do: Can I make this more generalizable and less hardcoded? 
        var1 = float(sample_from_distribution(self.data[scenario]['var1']['mean'], self.data[scenario]['var1']['stdev'], num_samples=1)[0])
        var2 = float(sample_from_distribution(self.data[scenario]['var2']['mean'], self.data[scenario]['var2']['stdev'], num_samples=1)[0])
        var3 = float(sample_from_distribution(self.data[scenario]['var3']['mean'], self.data[scenario]['var3']['stdev'], num_samples=1)[0])
        var4 = float(sample_from_distribution(self.data[scenario]['var4']['mean'], self.data[scenario]['var4']['stdev'], num_samples=1)[0])
        var5 = float(sample_from_distribution(self.data[scenario]['var5']['mean'], self.data[scenario]['var5']['stdev'], num_samples=1)[0])
        variables_used = {'var1': var1, "var2": var2, "var3": var3, "var4": var4, 'var5': var5}
        return variables_used, payoff_func(var1, var2, var3, var4, var5)

In [ ]:
def payoffs_formula(var1, var2, var3, var4, var5): 
    return var1+var2+var3+var4+var5

In [ ]:
def create_game(outcomes):
    # Create a new game where "China" is the root player
    game = Game()
    
    # Add moves: China chooses between "Tariff" or "No Tariff"
    game.add_moves(player="China", actions=["Tariff", "No Tariff"])
    
    # Add moves: The US responds to China's move
    game.add_moves(player="US", actions=["Tariff", "No Tariff"])
    
    game.add_outcomes(outcomes)

    return game

In [ ]:
player1 = Player(player1_data, "China")
player2 = Player(player2_data, "USA")
num_sims = 1000

tt = {} 
tnt = {} 
ntt = {} 
ntnt = {} 
results = {}

In [ ]:
for i in tqdm(range(num_sims)): 
    # Sample Player 1 Payoffs 
    p1_tt_variables, p1_tt = player1.calculate_payoff('tt', payoffs_formula)
    p1_tnt_variables, p1_tnt = player1.calculate_payoff('tnt', payoffs_formula)
    p1_ntt_variables, p1_ntt = player1.calculate_payoff('ntt', payoffs_formula)
    p1_ntnt_variables, p1_ntnt = player1.calculate_payoff('ntnt', payoffs_formula)

    player1_payoffs = {'tt': p1_tt, 'tnt': p1_tnt, 'ntt': p1_ntt, 'ntnt': p1_ntnt} 
    
    # Sample Player 2 payoffs
    p2_tt_variables, p2_tt = player2.calculate_payoff('tt', payoffs_formula)
    p2_tnt_variables, p2_tnt = player2.calculate_payoff('tnt', payoffs_formula)
    p2_ntt_variables, p2_ntt = player2.calculate_payoff('ntt', payoffs_formula)
    p2_ntnt_variables, p2_ntnt = player2.calculate_payoff('ntnt', payoffs_formula)
    
    player2_payoffs = {'tt': p2_tt, 'tnt': p2_tnt, 'ntt': p2_ntt, 'ntnt': p2_ntnt} 

    outcomes = [
        (int(p1_tt), int(p2_tt)),  # Both impose tariffs
        (int(p1_tnt), int(p2_tnt)),    # China tariffs, US does not
        (int(p1_ntt), int(p2_ntt)),   # China does not tariff, US does
        (int(p1_ntnt), int(p2_ntnt))   # Neither imposes tariffs
    ]

    tt[i] = {'player1': p1_tt_variables,
             'player2': p2_tt_variables}

    tnt[i] = {'player1': p1_tnt_variables,
             'player2': p2_tnt_variables}

    ntt[i] = {'player1': p1_ntt_variables,
             'player2': p2_ntt_variables}
    
    ntnt[i] = {'player1': p1_ntnt_variables,
             'player2': p2_ntnt_variables}

    g = create_game(outcomes)
    solver = BackwardInductionSolver(g)
    solver.solve()
    sim_result = solver.record_equilibrium() # What happens if there are two equilibria? 

    results[i] = {'player1': player1_payoffs,
                  'player2': player2_payoffs, 
                  'actions': sim_result
                 }

In [ ]:
def convert_dict_to_df(dictionary): 
    # First, let's create an empty list to store our data
    data = []
    
    # Iterate through each key-value pair in the outer dictionary
    for scenario, players in dictionary.items():
        # Create a row dictionary for this scenario
        row = {'simulation': scenario}
        
        # Iterate through each player
        for player, variables in players.items():
            # Iterate through each variable for this player
            for var_name, var_value in variables.items():
                # Create a column name by combining player and variable names
                column_name = f"{player}{var_name}"
                # Add this data to our row
                row[column_name] = var_value
        
        # Add the completed row to our data list
        data.append(row)
    
    # Create a DataFrame from our list of dictionaries
    df = pd.DataFrame(data)
    
    # Set the scenario column as the index
    df.set_index('simulation', inplace=True)
    return df

In [ ]:
tt_df = convert_dict_to_df(tt) 
tnt_df = convert_dict_to_df(tnt)
ntt_df = convert_dict_to_df(ntt)
ntnt_df = convert_dict_to_df(ntnt) 
results_df = convert_dict_to_df(results)

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:  
    tt_df.to_excel(writer, sheet_name='Tariff Tariff')
    tnt_df.to_excel(writer, sheet_name='Tariff No Tariff')
    ntt_df.to_excel(writer, sheet_name='No Tariff Tariff')
    ntnt_df.to_excel(writer, sheet_name='No Tariff No Tariff')
    results_df.to_excel(writer, sheet_name='Game Results')

## Summary of Game Results

In [ ]:
df = pd.read_excel('output.xlsx', sheet_name = 'Game Results')
col1 = 'actionsChina'
col2 = 'actionsUS'

tuples = list(zip(df[col1], df[col2]))
unique_tuples = list(set(tuples))

tuple_counts = {}
for tup in unique_tuples:
    tuple_counts[tup] = tuples.count(tup)

plt.figure(figsize=(10, 6))
labels = [f"{col1[7:]} {t[0]}, {col2[7:]} {t[1]}" for t in unique_tuples]
counts = list(tuple_counts.values())

sorted_indices = np.argsort(counts)[::-1]  # Descending order
sorted_labels = [labels[i] for i in sorted_indices]
sorted_counts = [counts[i] for i in sorted_indices]
bars = plt.bar(range(len(sorted_labels)), sorted_counts)
plt.xticks(range(len(sorted_labels)), sorted_labels, rotation=45, ha='right')
plt.xlabel('Unique Value Pairs')
plt.ylabel('Frequency')
plt.title(f'Frequency of Outcomes')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')
plt.tight_layout()
plt.show()

## Review Game Results

In [ ]:
simulation_number = 19

In [ ]:
print(f'TT outcomes is: ({df.loc[simulation_number]['player1tt']:.2f}, {df.loc[simulation_number]['player2tt']:.2f})')
print(f'TNT outcomes is: ({df.loc[simulation_number]['player1tnt']:.2f}, {df.loc[simulation_number]['player2tnt']:.2f})')
print(f'NTT outcomes is: ({df.loc[simulation_number]['player1ntt']:.2f}, {df.loc[simulation_number]['player2ntt']:.2f})')
print(f'NTNT outcomes is: ({df.loc[simulation_number]['player1ntnt']:.2f}, {df.loc[simulation_number]['player2ntnt']:.2f})')

In [ ]:
outcomes = [
        (np.round(df.loc[simulation_number]['player1tt'], 2), np.round(df.loc[simulation_number]['player2tt'],2)),  # Both impose tariffs
        (np.round(df.loc[simulation_number]['player1tnt'], 2), np.round(df.loc[simulation_number]['player2tnt'],2)),    # China tariffs, US does not
        (np.round(df.loc[simulation_number]['player1ntt'], 2), np.round(df.loc[simulation_number]['player2ntt'],2)),   # China does not tariff, US does
        (np.round(df.loc[simulation_number]['player1ntnt'], 2), np.round(df.loc[simulation_number]['player2ntnt'],2))   # Neither imposes tariffs
    ]

In [ ]:
g = create_game(outcomes)
solver = BackwardInductionSolver(g)
solver.solve()
solver.visualize_equilibrium()